# Numba_101

Numba gives you the power to speed up your applications with high performance functions written directly in Python. With a few annotations, array-oriented and math-heavy Python code can be just-in-time compiled to native machine instructions, similar in performance to C, C++ and Fortran, without having to switch languages or Python interpreters.

Numba works by generating optimized machine code using the LLVM compiler infrastructure at import time, runtime, or statically (using the included pycc tool). Numba supports compilation of Python to run on either CPU or GPU hardware, and is designed to integrate with the Python scientific software stack.




In [1]:
import numpy as np
from numba import jit

Let's first talk about some basic numpy array math.
Let's start with a random array and take the sum with the builtin `sum` function and our own function.

In [2]:
random_array = np.random.rand(100)

def python_sum(A):
    s = 0
    for i in range(A.shape[0]):
        s += A[i]
    return s
%timeit python_sum(random_array)
%timeit sum(random_array)
%timeit np.sum(random_array)

12.6 µs ± 171 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
7.27 µs ± 108 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
2.7 µs ± 62 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


We can see that the builtin `sum` method is much faster than our own sum method.
But what would happen if we decorate python_sum with numba?

In [ ]:
numba_sum = jit(python_sum)

```python
numba_sum = jit(python_sum)
```
is equivilent to
```python
@jit
def numba_sum():
    same logic
```

In [ ]:
%timeit numba_sum(random_array)
%timeit numba_sum(random_array)

We can see that there is about a 10x increase in speed using numba over np.sum

but we also had a very large overhead fron jit compilation. The first run was very slow, but the next 13 were very quick.

Numba is fantastic for numeric computation.
Numba does not work well with Strings or other Python objects (currently).

Numba jit has two modes of operation. `python` and `nopython`.
in `python` mode, python objects are used in the function
in `nopython` mode, native types are used in the function.
The default for jit is `nopython` mode, and `python` mode is used if the jit compiler can't compile to `nopython` mode

`nopython` mode is much faster so you can raise an exception if `nopython` mode can't be compiled with @jit(nopython=true) or the decorator @njit

In [ ]:
from numba import njit

@njit
def concat_strings(a,b):
    return a + b
concat_strings('a','b')

The jit compiler does not have support for strings or python objects so it raised the exception.

## Release the GIL
Numba jit can also be used to release the gil and do computation in paralell.
Let's make a 2d array and paralellize the sum.

Numba has a function `prange` it is it stands for parallel range and acts like Pythons builtin function `range` but tells the jit compiler to use multithreading.

In [ ]:
from numba import prange
random_array_2d = np.random.rand(10000, 40)

@njit
def sequential_sum(A):
    s = 0
    results = np.zeros(A.shape[1])
    for i in prange(A.shape[1]):
        results[i] = np.sum(A[:,i])
    return results

@njit(parallel=True, nogil=True)
def parallel_sum(A):
    s = 0
    results = np.zeros(A.shape[1])
    for i in prange(A.shape[1]):
        results[i] = np.sum(A[:,i])
    return results
# To show the outputs are the same. Let's print the results

print(sequential_sum(random_array_2d))
print(parallel_sum(random_array_2d))


Now let's time the differences

In [ ]:
%timeit sequential_sum(random_array_2d)
%timeit parallel_sum(random_array_2d)

So we can see that running it in parallel was actually faster.
But let's compare the speed to numpy sum

In [ ]:
%timeit np.sum(random_array_2d, axis=0)

## Conclusion
We can see that our Numba implementation was slower than NumPy.

The takeaway is that Numba can see performance gains by threading and releasing the GIL but NumPy is very fast and shouldn't be reinvented.

For code that NumPy doesn't have a function for, Numba can be a great tool for performance.